In [ ]:
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
import os

from scipy.optimize import curve_fit

from functionsFIMS import getAnalysisNumbers, plotPolyExamples
from runDataClass import runData
from polyaClass import myPolya

In [ ]:
if __name__ == '__main__':
    runNos = getAnalysisNumbers()
    simData = runData(1173)
    treenames = simData.getTreeNames()
    print(treenames)
    for name in treenames:
        simData.printColumns(name)

In [ ]:
if __name__ == '__main__':
    simData.plot2DFieldLines('Cathode')
    simData.plotAllFieldLines()

In [ ]:
test = simData.getDataFrame('metaData')

In [ ]:
print(test)

In [ ]:
plotPolyExamples()

In [ ]:
testPolya = myPolya(gain=60, theta=0.5)
testPolya.solveForGain(targetEff=.92, threshold=10)
print(testPolya.gain)

In [ ]:
print(np.arange(0, 3.1, 1))

In [ ]:
def requiredGain(theta, efficiency=.95, threshold=10):
    """
    TODO
    """
    from scipy.special import gammaincinv
    
    if not (0 < efficiency < 1):
        raise ValueError('Efficiency must be between 0 and 1.')
    if threshold <= 0:
        raise ValueError('Threshold must be positive.')

    alpha = theta + 1
    pLower = 1.0 - efficiency

    try:
        xArg = gammaincinv(alpha, pLower)
    except Exception as e:
        print(f"Error during gammaincinv calculation: {e}")
        raise

    nominalGain = (threshold * alpha) / xArg

    gainResult = np.where(xArg <= 0, np.inf, nominalGain)
    
    return gainResult

from scipy.optimize import curve_fit
def exponentialDecay(x, a, b, c):
    return a*np.exp(-b*x) + c

def fitExponentialDecay(thetaValues, gainValues):

    from scipy.stats import chi2 as chi2Dist

    popt, pcov = curve_fit(exponentialDecay, thetaValues, gainValues, p0=[100, 0.1, 1])
    perr = np.sqrt(np.diag(pcov))

    yFit = exponentialDecay(thetaValues, *popt)
    residuals = gainValues - yFit
    dof = len(thetaValues)-len(popt)
    chi2 = np.sum(residuals**2)
    rChi2 = chi2/dof
    pVal = chi2Dist.sf(chi2, dof)
    
    chi2Info = {
        'chi2': chi2, 
        'dof': dof,
        'rChi2': rChi2,
        'pVal': pVal
    }

    return popt, perr, chi2Info


def plotRequireGains():
    """
    TODO
    """

    theta = np.linspace(0, 2, 31)
    fitTheta = np.linspace(0,2,100)

    plotConfigs = [
        {
            'eff': 0.95, 'thresh': 10, 
            'c': 'b', 'marker': 'o'
        },
        {
            'eff': 0.95, 'thresh': 12, 
            'c': 'g', 'marker': 'x'
        },
        {
            'eff': 0.95, 'thresh': 8, 
            'c': 'r', 'marker': 'x'
        },
        {
            'eff': 0.94, 'thresh': 10, 
            'c': 'm', 'marker': '+'
        },
        {
            'eff': 0.96, 'thresh': 10, 
            'c': 'c', 'marker': '+'
        }
    ]

    nominalVals = plotConfigs[0]
    gainsNominal = requiredGain(theta, efficiency=nominalVals['eff'], threshold=nominalVals['thresh'])
    fitParam, fitParamErr, chi2Info = fitExponentialDecay(theta, gainsNominal) 
    

    fig = plt.figure(figsize=(12, 15))
    fig.suptitle('Gain Sensitivity')
    ax = fig.add_subplot(311)
    axNorm = fig.add_subplot(312)
    axNominal = fig.add_subplot(313)

    for inConfig in plotConfigs:
        
        inGain = requiredGain(theta, efficiency=inConfig['eff'], threshold=inConfig['thresh'])
        ax.scatter(
            theta, inGain,
            c=inConfig['c'], marker=inConfig['marker'],
            label=f'Efficiency = {inConfig['eff']}, Threshold = {inConfig['thresh']}'
        )
        axNorm.plot(
            theta, inGain/gainsNominal,
            c=inConfig['c'], marker=inConfig['marker'],
            label=f'Efficiency = {inConfig['eff']}, Threshold = {inConfig['thresh']}'
        )   

    fitResults = exponentialDecay(fitTheta, *fitParam)
    ax.plot(
            fitTheta, fitResults,
            c='b', ls='--',
            label=f'Exponential Fit (rChi2 = {chi2Info['rChi2']:.2f}, pVal = {chi2Info['pVal']:.1e})'
        )
    gainTheta0 = requiredGain(0, efficiency=.95, threshold=10)
    axNominal.scatter(
            theta, gainsNominal/gainTheta0,
            c='b', marker='o',
            label=f'Efficiency = {nominalVals['eff']}, Threshold = {nominalVals['thresh']}\nNormalized to gain at Theta = 0'
        )
    gainFitTheta0 = exponentialDecay(0, *fitParam)
    axNominal.plot(
            fitTheta, fitResults/gainFitTheta0,
            c='b', ls='--',
            label=f'Normalized Exponential Fit (rChi2 = {chi2Info['rChi2']:.2f}, pVal = {chi2Info['pVal']:.1e})'
        )


    ax.set_title('Gain vs. Polya Shape')
    ax.set_xlabel('Polya Shape (Theta)')
    ax.set_ylabel('Gas Gain')
    ax.grid()
    ax.legend()

    axNorm.set_title('Normalized Gain vs. Polya Shape')
    axNorm.set_xlabel('Polya Shape (Theta)')
    axNorm.set_ylabel('Normalized Gas Gain')
    axNorm.grid()
    axNorm.legend()

    axNominal.set_title('Normalized Gain vs. Polya Shape')
    axNominal.set_xlabel('Polya Shape (Theta)')
    axNominal.set_ylabel('Normalized Gas Gain')
    axNominal.grid()
    axNominal.legend()
    
    return

plotRequireGains()